## Import dependencies
Execute ‘bash dep_install.bash‘ in terminal

In [15]:
import pandas as pd
import matplotlib as mp
import numpy as np

Load temperature data set (https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)

In [16]:
global_temperature_data = pd.read_csv("./data/climate-change-earth-surface-temperature-data/GlobalTemperatures.csv")

In [ ]:
global_temperature_data

## Induktive Statistik

4. a) 95% Konfidenzintervall (Was sind die Grenzen für die Temperatur) ... '95% der Temperaturen liefen in folgendem Intervall'

b) Wie groß muss der Stichprobenumfang mindestens sein, damit die Breite des 9% Konfidenzintervalls für die Temperatur höchstens 20 *C

Signifikanz Test
